# Technical Indicators Trading Strategy

This notebook implements a trading strategy using various technical indicators to generate trading signals for stocks. We'll use popular indicators like Moving Averages, RSI, MACD, and Bollinger Bands to create a comprehensive trading system.

## 1. Setup Environment and Import Libraries

First, let's import all the necessary libraries we'll need for our analysis.

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import ta
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# Set pandas display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# Set plotting style
plt.style.use('seaborn')
%matplotlib inline

## 2. Load and Prepare Financial Data

Now we'll download historical price data for our analysis. We'll use the SPY ETF as an example.

In [ ]:
# Download SPY data
symbol = 'SPY'
start_date = '2020-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

# Download data using yfinance
df = yf.download(symbol, start=start_date, end=end_date)

# Calculate daily returns
df['Returns'] = df['Adj Close'].pct_change()

# Display the first few rows
df.head()

## 3. Calculate Technical Indicators

Let's calculate several technical indicators that we'll use in our trading strategy:
1. Simple Moving Averages (SMA)
2. Relative Strength Index (RSI)
3. MACD (Moving Average Convergence Divergence)
4. Bollinger Bands

In [ ]:
# Calculate technical indicators
def calculate_indicators(df):
    # Moving Averages
    df['SMA20'] = ta.trend.sma_indicator(df['Close'], window=20)
    df['SMA50'] = ta.trend.sma_indicator(df['Close'], window=50)
    
    # RSI
    df['RSI'] = ta.momentum.rsi(df['Close'], window=14)
    
    # MACD
    macd = ta.trend.MACD(df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    df['MACD_Hist'] = macd.macd_diff()
    
    # Bollinger Bands
    bollinger = ta.volatility.BollingerBands(df['Close'])
    df['BB_Upper'] = bollinger.bollinger_hband()
    df['BB_Middle'] = bollinger.bollinger_mavg()
    df['BB_Lower'] = bollinger.bollinger_lband()
    
    return df

# Apply indicators to our dataset
df = calculate_indicators(df)

# Display the first few rows with indicators
df.tail()

## 4. Create Trading Signals

Now we'll create trading signals based on our technical indicators. We'll use the following rules:
1. Buy when:
   - RSI < 30 (oversold)
   - MACD crosses above signal line
   - Price crosses above SMA50
2. Sell when:
   - RSI > 70 (overbought)
   - MACD crosses below signal line
   - Price crosses below SMA50

In [ ]:
# Generate trading signals
def generate_signals(df):
    # Initialize signals
    df['Signal'] = 0
    
    # RSI Signals
    df.loc[df['RSI'] < 30, 'RSI_Signal'] = 1  # Oversold
    df.loc[df['RSI'] > 70, 'RSI_Signal'] = -1  # Overbought
    
    # MACD Signals
    df['MACD_Cross'] = 0
    df.loc[df['MACD'] > df['MACD_Signal'], 'MACD_Cross'] = 1
    df.loc[df['MACD'] < df['MACD_Signal'], 'MACD_Cross'] = -1
    
    # Moving Average Signals
    df['MA_Cross'] = 0
    df.loc[df['Close'] > df['SMA50'], 'MA_Cross'] = 1
    df.loc[df['Close'] < df['SMA50'], 'MA_Cross'] = -1
    
    # Combine Signals
    df['Signal'] = (df['RSI_Signal'] + df['MACD_Cross'] + df['MA_Cross']).fillna(0)
    df['Position'] = df['Signal'].replace(to_replace=0, method='ffill')
    
    return df

# Apply signals to our dataset
df = generate_signals(df)

# Display the last few rows with signals
df.tail()

## 5. Implement Trading Strategy

Let's implement our trading strategy and calculate returns based on our signals.

In [ ]:
# Calculate strategy returns
def calculate_strategy_returns(df):
    # Calculate strategy returns
    df['Strategy_Returns'] = df['Returns'] * df['Position'].shift(1)
    
    # Calculate cumulative returns
    df['Cumulative_Market_Returns'] = (1 + df['Returns']).cumprod()
    df['Cumulative_Strategy_Returns'] = (1 + df['Strategy_Returns']).cumprod()
    
    return df

# Apply strategy calculations
df = calculate_strategy_returns(df)

# Display final results
print("\nStrategy Performance Summary:")
print(f"Total Market Returns: {(df['Cumulative_Market_Returns'].iloc[-1] - 1)*100:.2f}%")
print(f"Total Strategy Returns: {(df['Cumulative_Strategy_Returns'].iloc[-1] - 1)*100:.2f}%")

## 6. Visualize Results

Let's create some visualizations to analyze our strategy's performance.

In [ ]:
# Create interactive plot using plotly
def plot_strategy_results(df):
    fig = make_subplots(rows=4, cols=1, 
                        shared_xaxes=True,
                        vertical_spacing=0.03,
                        row_heights=[0.4, 0.2, 0.2, 0.2])

    # Price and Moving Averages
    fig.add_trace(go.Candlestick(x=df.index,
                                open=df['Open'],
                                high=df['High'],
                                low=df['Low'],
                                close=df['Close'],
                                name='OHLC'),
                 row=1, col=1)
    
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA20'],
                            name='SMA20',
                            line=dict(color='blue', width=1)),
                 row=1, col=1)
    
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA50'],
                            name='SMA50',
                            line=dict(color='red', width=1)),
                 row=1, col=1)

    # RSI
    fig.add_trace(go.Scatter(x=df.index, y=df['RSI'],
                            name='RSI',
                            line=dict(color='purple', width=1)),
                 row=2, col=1)
    
    # Add RSI levels
    fig.add_hline(y=70, line_width=1, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_width=1, line_dash="dash", line_color="green", row=2, col=1)

    # MACD
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD'],
                            name='MACD',
                            line=dict(color='blue', width=1)),
                 row=3, col=1)
    
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD_Signal'],
                            name='Signal',
                            line=dict(color='orange', width=1)),
                 row=3, col=1)

    # Strategy Performance
    fig.add_trace(go.Scatter(x=df.index, y=df['Cumulative_Market_Returns'],
                            name='Buy & Hold',
                            line=dict(color='blue', width=1)),
                 row=4, col=1)
    
    fig.add_trace(go.Scatter(x=df.index, y=df['Cumulative_Strategy_Returns'],
                            name='Strategy',
                            line=dict(color='green', width=1)),
                 row=4, col=1)

    # Update layout
    fig.update_layout(
        title='Technical Trading Strategy Analysis',
        xaxis_title='Date',
        height=1200,
        showlegend=True,
        xaxis_rangeslider_visible=False
    )

    fig.show()

# Plot the results
plot_strategy_results(df)